In [1]:
import pandas as pd

import import_ipynb
from preprocessor import preprocessor, drop_index, read_file

result_path = '../result_kor_OSX.csv'
result_out_path = 'result_out.csv'

def predict(condition, model):
    target_columns = condition.target_columns
    for target_column in target_columns:
        dropping_columns = target_columns.drop(target_column)
        X, y = propeocessor.preprocessed(target_column, dropping_columns)
        model.train(X, y, target_column, dropping_columns)
    return pd.Series(dict(
        (target_column, model.predict(condition.preprocessed, target_column, dropping_columns))
        for target_column, model in zip(target_columns, models)))

def char_to_int(char):
    return ord(char.lower()) - 96

model = Model()
completed_rows = []
for condition in preprocessor.conditions:
    original = drop_index(condition.original.dropna(), condition.target_columns)
    predicted = preprocessor.restore_predicted(predict(condition, model))
    completed_row = pd.concat([original, predicted])
    completed_rows.append(completed_row)
completed = (pd.concat(completed_rows, axis=1, sort=False).T
               .reindex(columns=preprocessor.predict_columns_order))
completed['사상자수'] = (completed.reindex(columns=['사망자수', '중상자수', '경상자수'])
                                .apply(sum, axis=1))
result = read_file(result_path)

rows = []
for index, row in result.iterrows():
    answer = completed.loc[row['열'] - 2][char_to_int(row['행']) - 1]
    row['값'] = answer
    rows.append(row)
result_out = pd.DataFrame(rows)
result_out.to_csv(result_out_path)

importing Jupyter notebook from preprocessor.ipynb
